In [2]:
import os
import csv
import logging
import ee
import urllib.request
import numpy as np
import rasterio
import shutil
from pathlib import Path
from PIL import Image
import yaml
from dotenv import load_dotenv
from ee import ServiceAccountCredentials

# ─────────────────────────── Load environment variables ───────────────────────────
load_dotenv()

# ─────────────────────────── Load configuration from config.yaml ───────────────────────────
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

RAW_DATA_DIR = Path(config["raw_data_dir"])
RESULTS_DIR  = Path(config["results_dir"])

# Directory containing the LiDAR DTM tiles
DTM_DIR = (
    RAW_DATA_DIR
    / "datasets"
    / "nasa-amazon-lidar-2008-2018"
    / "Nasa_lidar_2008_to_2018_DTMs"
    / "DTM_tiles"
)

# CSV file listing the candidate tiles to process
PATH_TO_COORDS_CSV = RESULTS_DIR / "candidates_top500.csv"

# Output directory for predicted results
OUT_DIR = RESULTS_DIR / "predicted"
# Create output directory if it doesn't exist
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ─────────────────────────── Earth Engine authentication ───────────────────────────
GSA_EMAIL_VAR = os.getenv("GSA_EMAIL")
KEY_PATH_VAR  = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

creds = ServiceAccountCredentials(GSA_EMAIL_VAR, KEY_PATH_VAR)
ee.Initialize(credentials=creds, project=os.getenv("GEE_PROJECT", "kaggle-ai-to-z"))

# ─────────────────────────── Processing parameters ───────────────────────────
MAX_PHOTOS_PER_PREFIX = 3       # Max entries per prefix
BUFFER_RADIUS_METERS  = 1500    # Radius for buffering points (meters)
DATE_TO_DOWNLOAD      = "2025-05-01"
AZ1, ALT1             = 315, 45  # Azimuth and altitude for first hillshade
AZ2, ALT2             = 45, 30   # Azimuth and altitude for second hillshade

# Sentinel-1 and Sentinel-2 collection and visualization parameters
EE_COLLECTION_S1 = "COPERNICUS/S1_GRD"
VIS_PARAMS_S1    = {'bands': ['VV'], 'min': -25, 'max': 5}
EE_COLLECTION_S2 = "COPERNICUS/S2_SR_HARMONIZED"
VIS_PARAMS_S2    = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.3}

# Ensure the CSV exists
if not PATH_TO_COORDS_CSV.exists():
    raise FileNotFoundError(f"CSV not found: {PATH_TO_COORDS_CSV}")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)

# ─────────────────────────── Helper functions ───────────────────────────
def get_best_s1(pt, start, end):
    """
    Fetch the earliest Sentinel-1 image in the given date range for the point.
    Returns the image and its bounding geometry.
    """
    geom = ee.Geometry.Point(pt).buffer(BUFFER_RADIUS_METERS).bounds()
    img  = (
        ee.ImageCollection(EE_COLLECTION_S1)
        .filterBounds(ee.Geometry.Point(pt))
        .filterDate(start, end)
        .sort('system:time_start')
        .first()
    )
    return img, geom

def get_best_s2(pt, start, end, max_cloud=30):
    """
    Fetch the least cloudy Sentinel-2 image in the given date range for the point.
    Returns the image and its bounding geometry.
    """
    geom = ee.Geometry.Point(pt).buffer(BUFFER_RADIUS_METERS).bounds()
    img  = (
        ee.ImageCollection(EE_COLLECTION_S2)
        .filterBounds(ee.Geometry.Point(pt))
        .filterDate(start, end)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud))
        .sort('CLOUDY_PIXEL_PERCENTAGE')
        .first()
    )
    return img, geom

def save_jpg(img, region, out_path, vis_params):
    """
    Save a visualized Earth Engine image as JPEG.
    """
    vis = img.visualize(**vis_params)
    url = vis.getThumbURL({'region': region, 'dimensions': 800, 'format': 'jpg'})
    data = urllib.request.urlopen(url).read()
    out_path.write_bytes(data)

def save_tif(img, region, out_path, bands=None):
    """
    Save an Earth Engine image as GeoTIFF.
    """
    params = {'scale': 10, 'region': region, 'format': 'GEO_TIFF', 'crs': 'EPSG:4326'}
    if bands:
        params['bands'] = bands
    url  = img.getDownloadURL(params)
    data = urllib.request.urlopen(url).read()
    out_path.write_bytes(data)

def hillshade(arr, az, alt):
    """
    Compute hillshade from a single-band array given sun azimuth and altitude.
    """
    az, alt = np.deg2rad([az, alt])
    dy, dx  = np.gradient(arr.astype("float32"), edge_order=2)
    slope   = np.arctan(np.hypot(dx, dy))
    aspect  = np.arctan2(dy, -dx)
    hs = (np.sin(alt) * np.cos(slope) +
          np.cos(alt) * np.sin(slope) * np.cos(az - aspect))
    return (np.clip(hs, 0, 1) * 255).astype("uint8")

# ─────────────────────────── Main processing ───────────────────────────
if __name__ == "__main__":
    # Clear the output directory before running
    if OUT_DIR.exists():
        shutil.rmtree(OUT_DIR)
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    matches = []
    prefix_counts = {}

    # Read candidate tiles from CSV and limit by prefix
    with open(PATH_TO_COORDS_CSV, newline='', encoding='utf-8') as cf:
        reader = csv.DictReader(cf)
        for row in reader:
            full_stem = Path(row["filename"]).stem
            prefix    = "_".join(full_stem.split("_")[:2])
            cnt       = prefix_counts.get(prefix, 0)
            if cnt >= MAX_PHOTOS_PER_PREFIX:
                continue
            prefix_counts[prefix] = cnt + 1

            # Compute center coordinates of the tile
            minx, miny = float(row["min_lon"]), float(row["min_lat"])
            maxx, maxy = float(row["max_lon"]), float(row["max_lat"])
            center     = [(minx + maxx) / 2, (miny + maxy) / 2]
            matches.append((full_stem, center))

    logging.info(
        f"Found {len(matches)} entries across {len(prefix_counts)} prefixes (max {MAX_PHOTOS_PER_PREFIX} each)"
    )

    # Process each tile
    for stem, pt in matches:
        logging.info(f"Processing tile {stem}")
        tile_dir = OUT_DIR / stem
        tile_dir.mkdir(exist_ok=True)

        # Copy LiDAR DTM file
        lidar_fp = DTM_DIR / f"{stem}.tif"
        if not lidar_fp.exists():
            logging.warning(f"LiDAR TIFF missing for {stem}")
            continue
        shutil.copy(lidar_fp, tile_dir / f"{stem}_lidar.tif")

        # Define date range and region for Earth Engine
        start  = "2024-01-01"
        end    = f"{DATE_TO_DOWNLOAD}T23:59:59"
        region = ee.Geometry.Point(pt).buffer(BUFFER_RADIUS_METERS).bounds()

        # Fetch and save Sentinel-1 imagery
        s1_img, s1_reg = get_best_s1(pt, start, end)
        if s1_img:
            save_jpg(s1_img, s1_reg, tile_dir / f"{stem}_S1_{DATE_TO_DOWNLOAD}.jpg", VIS_PARAMS_S1)
            save_tif(s1_img, s1_reg, tile_dir / f"{stem}_S1_{DATE_TO_DOWNLOAD}.tif")
        else:
            logging.warning(f"No S1 imagery for {stem}")

        # Fetch and save Sentinel-2 imagery
        s2_img, s2_reg = get_best_s2(pt, start, end)
        if s2_img:
            save_jpg(s2_img, s2_reg, tile_dir / f"{stem}_S2_{DATE_TO_DOWNLOAD}.jpg", VIS_PARAMS_S2)
            save_tif(s2_img, s2_reg, tile_dir / f"{stem}_S2_{DATE_TO_DOWNLOAD}.tif", bands=['B4', 'B3', 'B2'])
        else:
            logging.warning(f"No S2 imagery for {stem}")

        # Compute and save hillshade composite from LiDAR
        with rasterio.open(lidar_fp) as src:
            arr = src.read(1).astype("float32")
            nod = src.nodata
            if nod is not None:
                arr[arr == nod] = np.nan
        hs1 = hillshade(arr, AZ1, ALT1)
        hs2 = hillshade(arr, AZ2, ALT2)
        comp = np.concatenate([hs1, hs2], axis=1)
        Image.fromarray(comp).save(tile_dir / f"{stem}_lidar_hillshade.jpg", quality=90)

    logging.info("Processing complete.")

2025-06-29 11:19:25,381 INFO Found 30 entries across 18 prefixes (max 3 each)
2025-06-29 11:19:25,382 INFO Processing tile ANT_A01_2011_laz_3
/tmp/ipykernel_17/4035491276.py:134: RuntimeWarning: invalid value encountered in cast
  return (np.clip(hs, 0, 1) * 255).astype("uint8")
2025-06-29 11:19:36,303 INFO Processing tile ANTL5770C8970_cleaned
2025-06-29 11:19:48,404 INFO Processing tile ANT_A01_2011_laz_5
2025-06-29 11:19:58,279 INFO Processing tile ANTL5780C8970_cleaned
2025-06-29 11:20:08,421 INFO Processing tile ANTL5790C8970_cleaned
2025-06-29 11:20:18,552 INFO Processing tile ANT_A01_2011_laz_8
2025-06-29 11:20:26,821 INFO Processing tile ANTL5800C8970_cleaned
2025-06-29 11:20:35,893 INFO Processing tile ANTL5770C8971_cleaned
2025-06-29 11:20:45,551 INFO Processing tile ANTL5780C8971_cleaned
2025-06-29 11:20:55,872 INFO Processing tile ANTL5790C8971_cleaned
2025-06-29 11:21:06,577 INFO Processing tile ANTL5800C8971_cleaned
2025-06-29 11:21:17,222 INFO Processing tile ANTL5770C89